In [1]:
import pandas as pd
import sys
from tqdm import tqdm
import time
from multiprocessing import Pool, cpu_count
import numpy as np
import os
from openai import OpenAI

sys.path.append('../functions')
from openai_law_matcher import apply_openai_law_matching, filter_official_regex_matches

In [2]:
# Verificar que la API key está configurada
api_key = os.getenv('OPEN_API_KEY')
# Inicializar cliente de OpenAI
client = OpenAI(api_key=api_key)

In [3]:
df = pd.read_csv('../../data/03_extracted/legal_docs_generic.csv')
df = df[~df['art_id'].str.contains('_TRANS', na=False)]
df['entity_label'].unique()

array(['LAW_MENTION', 'REGULATION', 'LAW_CODE', 'NOM'], dtype=object)

In [4]:
law_codes = df[df['entity_label'] == 'LAW_CODE']
law_codes.head()

,doc_id,art_id,entity_text,entity_label,pattern_group,full_context,words_before_count,words_after_count
26,2833199A,2833199A_2,Código de Instituciones y Procedimientos Elect...,LAW_CODE,LEGAL_DOCS,Para los efectos de este Reglamento se entende...,29,30
28,2833199A,2833199A_15,Código para las sesiones ordinarias y extraord...,LAW_CODE,LEGAL_DOCS,Los Consejeros Presidentes Distritales deberán...,29,30
68,3F66D749,3F66D749_1,Código de Instituciones y Procedimientos Elect...,LAW_CODE,LEGAL_DOCS,", dirigida a influir de manera negativa en las...",30,30
77,3F66D749,3F66D749_8,Código para la confección o elaboración de pro...,LAW_CODE,LEGAL_DOCS,y campaña ; Colocar propaganda en lugares expr...,30,30
78,3F66D749,3F66D749_10,Código para la confección o elaboración de pro...,LAW_CODE,LEGAL_DOCS,disposiciones administrativas y de protección ...,30,30


In [5]:
law_mentions_sin_match, law_mentions_con_match, stats_filtrado = filter_official_regex_matches(
    df=law_codes,
    entity_text_col='entity_text'
)

print(f"Filtrado completado")
print(f"Menciones CON match en regex: {stats_filtrado['con_match_regex']}")
print(f"Menciones SIN match (para LLM): {stats_filtrado['sin_match_regex']}")



=== FILTRANDO MENCIONES CON REGEX OFICIALES ===
Total de menciones a analizar: 231
Total de patrones regex a aplicar: 784
  Procesados: 4000/231
  Procesados: 6600/231

=== RESULTADOS DEL FILTRADO ===
Total de menciones: 231
Con match en regex (filtradas): 58
Sin match en regex (para LLM): 173
Porcentaje filtrado: 25.1%

=== EJEMPLOS DE MENCIONES FILTRADAS (YA TIENEN MATCH) ===
69. Código de Instituciones y Procedimientos Electorales de la Ciudad de México
   Match: CÓDIGO DE INSTITUCIONES Y PROCEDIMIENTOS ELECTORALES DE LA CIUDAD DE MÉXICO
95. Código de Instituciones y Procedimientos Electorales de la Ciudad de México
   Match: CÓDIGO DE INSTITUCIONES Y PROCEDIMIENTOS ELECTORALES DE LA CIUDAD DE MÉXICO
304. Código de Procedimientos Civiles para el Distrito Federal y demás ordenamientos 
   Match: CÓDIGO DE PROCEDIMIENTOS CIVILES PARA EL DISTRITO FEDERAL
438. Código de Procedimientos Civiles para el Distrito Federal
   Match: CÓDIGO DE PROCEDIMIENTOS CIVILES PARA EL DISTRITO FEDERAL
76

In [6]:
# Leer reglamentos oficiales
# Filtrar solo los reglamentos
leyes_completas = pd.read_csv('../../data/02_catalogs/leyes_hash.csv')
leyes_completas.head()
codigos_oficial = leyes_completas[leyes_completas['nombre_normalized'].str.startswith('CODIGO', na=False)]
codigos_oficial

,nombre,fecha_publicacion,fecha_actualizacion,link_pdf,link_docx,source_url,gov_level,nombre_normalized,doc_id
337,Codigo de Instituciones y Procedimientos Elect...,07/06/17,23/12/24,https://data.consejeria.cdmx.gob.mx/images/ley...,https://data.consejeria.cdmx.gob.mx/images/ley...,https://data.consejeria.cdmx.gob.mx/index.php/...,CDMX,CODIGODEINSTITUCIONESYPROCEDIMIENTOSELECTORALE...,98C6B489
338,Codigo de Etica de la Administracion Publica d...,07/02/19,NaN,https://data.consejeria.cdmx.gob.mx/images/ley...,https://data.consejeria.cdmx.gob.mx/images/ley...,https://data.consejeria.cdmx.gob.mx/index.php/...,CDMX,CODIGODEETICADELAADMINISTRACIONPUBLICADELACIUD...,AC512D6A
339,Codigo de Responsabilidad parlamentaria del Co...,03/04/19,NaN,https://data.consejeria.cdmx.gob.mx/images/ley...,https://data.consejeria.cdmx.gob.mx/images/ley...,https://data.consejeria.cdmx.gob.mx/index.php/...,CDMX,CODIGODERESPONSABILIDADPARLAMENTARIADELCONGRES...,9BCBA905
340,Codigo de Procedimientos Civiles para el Distr...,21/09/32,18/07/18,https://data.consejeria.cdmx.gob.mx/images/ley...,https://data.consejeria.cdmx.gob.mx/images/ley...,https://data.consejeria.cdmx.gob.mx/index.php/...,CDMX,CODIGODEPROCEDIMIENTOSCIVILESPARAELDISTRITOFED...,35C270ED
341,Codigo Civil para el Distrito Federal,26/05/26,29/11/24,https://data.consejeria.cdmx.gob.mx/images/ley...,https://data.consejeria.cdmx.gob.mx/images/ley...,https://data.consejeria.cdmx.gob.mx/index.php/...,CDMX,CODIGOCIVILPARAELDISTRITOFEDERAL,2F07FC1B
342,Codigo Fiscal de la Ciudad de Mexico,29/12/09,30/05/25,https://data.consejeria.cdmx.gob.mx/images/ley...,https://data.consejeria.cdmx.gob.mx/images/ley...,https://data.consejeria.cdmx.gob.mx/index.php/...,CDMX,CODIGOFISCALDELACIUDADDEMEXICO,04ABBAED
343,Codigo Penal para el Distrito Federal,16/07/02,30/05/25,https://data.consejeria.cdmx.gob.mx/images/ley...,https://data.consejeria.cdmx.gob.mx/images/ley...,https://data.consejeria.cdmx.gob.mx/index.php/...,CDMX,CODIGOPENALPARAELDISTRITOFEDERAL,FC3F5431
344,Codigo de Etica de los Servidores Publicos par...,09/07/14,NaN,https://data.consejeria.cdmx.gob.mx/images/ley...,https://data.consejeria.cdmx.gob.mx/images/ley...,https://data.consejeria.cdmx.gob.mx/index.php/...,CDMX,CODIGODEETICADELOSSERVIDORESPUBLICOSPARAELDIST...,13D9A1E2
345,Codigo Civil Federal,26/05/28,17/01/24,NaN,NaN,NaN,FEDERAL,CODIGOCIVILFEDERAL,C384FA60
346,Codigo de Comercio,07/10/89,30/12/24,NaN,NaN,NaN,FEDERAL,CODIGODECOMERCIO,CF38AF5C


In [7]:
# Aplicar matching con OpenAI solo a las menciones SIN match en regex
results_df_llm = apply_openai_law_matching(
    mentions_df=law_codes,
    cdmx_laws_df=codigos_oficial,
    client=client,
    entity_text_col='entity_text',
    art_id_col='art_id',
    cdmx_name_col='nombre',
    batch_size=10,
    delay_seconds=1,  # Espera 1 segundo entre cada llamada
    temperature=0.2   # Baja temperatura para resultados más consistentes
)

print(f"\nMatching con LLM completado. Total de resultados: {len(results_df_llm)}")

=== MATCHING DE LEYES CON OPENAI ===
Menciones a procesar: 231
Leyes oficiales CDMX: 17
Tiempo de espera entre llamadas: 1 segundos
Temperatura del modelo: 0.2

Filas válidas para procesar: 231 (de 231 total)
Tiempo estimado total: 3.9 minutos

Procesando todas las filas válidas basándose en art_id y entity_text...
Procesando lote 1/24
  Procesando art_id 2833199A_2: Código de Instituciones y Procedimientos Electoral...
  Procesando art_id 2833199A_15: Código para las sesiones ordinarias y extraordinar...
  Procesando art_id 3F66D749_1: Código de Instituciones y Procedimientos Electoral...
  Procesando art_id 3F66D749_8: Código para la confección o elaboración de propaga...
  Procesando art_id 3F66D749_10: Código para la confección o elaboración de propaga...
  Procesando art_id 3F66D749_10BIS: Código para la confección o elaboración de propaga...
  Procesando art_id 3F66D749_18: Código para las y los ministros de culto...
  Procesando art_id 3F66D749_102: Código de Instituciones y Pro

In [8]:
results_df_llm.to_csv('../../data/04_matched/code_mentions_matched.csv', index=False)